# 💬 Building Chat Applications with AI Beast

This tutorial covers building interactive chat applications using AI Beast's LLM module.

## What You'll Learn

1. Use the unified LLM interface
2. Manage conversation history
3. Build a chat loop
4. Add system prompts
5. Handle multiple providers

## Prerequisites

- Completed [01_getting_started.ipynb](01_getting_started.ipynb)
- At least one LLM model available

## 1. The Unified LLM Interface

AI Beast provides a unified interface for different LLM providers.

In [ ]:
import sys
sys.path.insert(0, '..')

from modules.llm import LLMClient

# Initialize with default provider (Ollama)
llm = LLMClient()
print(f"Using provider: {llm.provider}")

In [ ]:
# Simple chat
response = await llm.chat(
    messages=[
        {"role": "user", "content": "Hello! Who are you?"}
    ]
)

print(response.content)

## 2. Managing Conversation History

Keep track of conversation context for multi-turn chats.

In [ ]:
from dataclasses import dataclass, field
from typing import List, Dict

@dataclass
class Conversation:
    """Manages chat conversation history."""
    
    system_prompt: str = "You are a helpful AI assistant."
    messages: List[Dict[str, str]] = field(default_factory=list)
    max_history: int = 20
    
    def add_user(self, content: str) -> None:
        """Add a user message."""
        self.messages.append({"role": "user", "content": content})
        self._trim_history()
    
    def add_assistant(self, content: str) -> None:
        """Add an assistant message."""
        self.messages.append({"role": "assistant", "content": content})
        self._trim_history()
    
    def get_messages(self) -> List[Dict[str, str]]:
        """Get all messages including system prompt."""
        return [
            {"role": "system", "content": self.system_prompt},
            *self.messages
        ]
    
    def clear(self) -> None:
        """Clear conversation history."""
        self.messages.clear()
    
    def _trim_history(self) -> None:
        """Keep only recent messages."""
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history:]

# Create a conversation
convo = Conversation(system_prompt="You are a friendly Python tutor.")
print("Conversation manager created!")

In [ ]:
# Use the conversation
async def chat(user_input: str) -> str:
    """Send a message and get response."""
    convo.add_user(user_input)
    
    response = await llm.chat(
        messages=convo.get_messages()
    )
    
    convo.add_assistant(response.content)
    return response.content

# Test multi-turn conversation
print("User: What is a list in Python?")
response = await chat("What is a list in Python?")
print(f"Assistant: {response}\n")

print("User: How do I add items to it?")
response = await chat("How do I add items to it?")  # "it" refers to list from context
print(f"Assistant: {response}")

## 3. Interactive Chat Loop

Build a reusable chat interface.

In [ ]:
async def interactive_chat(
    system_prompt: str = "You are a helpful assistant.",
    model: str = "llama3.2",
    max_turns: int = 10,
):
    """Run an interactive chat session."""
    
    convo = Conversation(system_prompt=system_prompt)
    
    print("="*50)
    print(f"Chat session started (model: {model})")
    print("Type 'quit' to exit, 'clear' to reset")
    print("="*50 + "\n")
    
    for turn in range(max_turns):
        # Get user input
        user_input = input("You: ").strip()
        
        if not user_input:
            continue
        
        if user_input.lower() == 'quit':
            print("\nGoodbye!")
            break
        
        if user_input.lower() == 'clear':
            convo.clear()
            print("\n[Conversation cleared]\n")
            continue
        
        convo.add_user(user_input)
        
        # Stream response
        print("\nAssistant: ", end="", flush=True)
        
        full_response = ""
        async for chunk in llm.chat_stream(
            messages=convo.get_messages(),
            model=model,
        ):
            print(chunk.content, end="", flush=True)
            full_response += chunk.content
        
        print("\n")
        convo.add_assistant(full_response)
    
    return convo.messages

# Run interactive chat (uncomment to use)
# await interactive_chat(
#     system_prompt="You are a friendly coding assistant.",
#     model="llama3.2"
# )

## 4. Effective System Prompts

System prompts shape the assistant's behavior.

In [ ]:
# Different personas with system prompts

SYSTEM_PROMPTS = {
    "coder": """You are an expert programmer. 
- Always provide code examples
- Explain concepts clearly
- Follow best practices
- Consider edge cases""",
    
    "teacher": """You are a patient teacher.
- Break down complex topics
- Use simple analogies
- Ask guiding questions
- Encourage learning""",
    
    "reviewer": """You are a code reviewer.
- Identify potential bugs
- Suggest improvements
- Check security issues
- Be constructive""",
    
    "creative": """You are a creative writer.
- Be imaginative
- Use vivid descriptions
- Tell engaging stories
- Have fun with ideas"""
}

# Test different personas
query = "What is recursion?"

for persona, prompt in list(SYSTEM_PROMPTS.items())[:2]:  # Test first 2
    print(f"\n{'='*50}")
    print(f"Persona: {persona}")
    print("="*50)
    
    response = await llm.chat(
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": query}
        ]
    )
    
    print(response.content[:500] + "..." if len(response.content) > 500 else response.content)

## 5. Structured Output

Guide the model to return structured data.

In [ ]:
import json

# Request JSON output
prompt = """Analyze this text and return JSON with sentiment and keywords.

Text: "I absolutely loved this product! It works amazingly well and 
the customer service was fantastic. Highly recommend to everyone."

Return ONLY valid JSON with this structure:
{
  "sentiment": "positive" | "negative" | "neutral",
  "confidence": 0.0-1.0,
  "keywords": ["keyword1", "keyword2"],
  "summary": "brief summary"
}"""

response = await llm.chat(
    messages=[{"role": "user", "content": prompt}],
    options={"temperature": 0.1}  # Lower temperature for consistency
)

print("Raw response:")
print(response.content)

# Parse JSON
try:
    # Try to extract JSON from response
    content = response.content
    if "```json" in content:
        content = content.split("```json")[1].split("```")[0]
    elif "```" in content:
        content = content.split("```")[1].split("```")[0]
    
    data = json.loads(content.strip())
    print("\nParsed JSON:")
    print(json.dumps(data, indent=2))
except json.JSONDecodeError as e:
    print(f"\nCouldn't parse JSON: {e}")

## 6. Multiple Providers

Switch between different LLM providers seamlessly.

In [ ]:
from modules.llm import LLMClient, LLMProvider

# Create clients for different providers
clients = {}

# Ollama (local)
try:
    clients["ollama"] = LLMClient(provider=LLMProvider.OLLAMA)
    print("✓ Ollama client ready")
except Exception as e:
    print(f"✗ Ollama unavailable: {e}")

# OpenAI (requires API key)
try:
    clients["openai"] = LLMClient(provider=LLMProvider.OPENAI)
    print("✓ OpenAI client ready")
except Exception as e:
    print(f"✗ OpenAI unavailable: {e}")

# Anthropic (requires API key)
try:
    clients["anthropic"] = LLMClient(provider=LLMProvider.ANTHROPIC)
    print("✓ Anthropic client ready")
except Exception as e:
    print(f"✗ Anthropic unavailable: {e}")

In [ ]:
# Compare responses across providers
prompt = "Explain machine learning in 2 sentences."

for name, client in clients.items():
    print(f"\n--- {name.upper()} ---")
    try:
        response = await client.chat(
            messages=[{"role": "user", "content": prompt}]
        )
        print(response.content)
    except Exception as e:
        print(f"Error: {e}")

## Next Steps

Great work! You've learned to build chat applications.

Continue with:
- **[03_rag_basics.ipynb](03_rag_basics.ipynb)** - Add document knowledge to your chats
- **[04_advanced_prompting.ipynb](04_advanced_prompting.ipynb)** - Master prompt engineering

## Exercises

1. Build a chatbot for a specific domain (cooking, fitness, etc.)
2. Add conversation summarization for long chats
3. Implement a multi-model chat that uses different models for different tasks